# Dependensies
`pip install 'stable-baselines3[extra]'`

In [1]:
import os
import gym # simulation environment
from stable_baselines3 import PPO # training algorithm
from stable_baselines3.common.vec_env import DummyVecEnv # if env.step() is fast, use this
# from stable_baselines3.common.vec_env import SubprocVecEnv # if env.step() is slow, use this
from stable_baselines3.common.evaluation import evaluate_policy 
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

# Environment setup

In [2]:
env_name = "CartPole-v0" # specify the environment
env_name

'CartPole-v0'

In [3]:
env = gym.make(env_name) # initialize the entire environment

In [4]:
episodes = 5 # epoch, times of training
for episode in range(episodes):
    state = env.reset() # initialize the state of environment
    done = False # is the environment done or not (e.g. clear the stage or the character is dead)
    score = 0 # total reward
    
    while not done:
        env.render() # visualize the environment
        action = env.action_space.sample() # random action
        state, reward, done, info = env.step(action) # take the action, and obtain the current state
        score += reward
    print(f'episode: {episode} score: {score}')
    env.close() # close the visualize window, but it doesn't work well

episode: 0 score: 14.0
episode: 1 score: 18.0
episode: 2 score: 24.0
episode: 3 score: 23.0
episode: 4 score: 16.0


# What's Environment ?
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py
## Types of space
- Box: range of values, n dimensional tensor
    - Box(0, 1, shape=(2,))
    - [0.1, 0.2], [0.25, 0.75], ...
- Discrete: set of items
    - Discrete(2)
    - 0, 1
- Tuple: tuple of other spaces
    - Tuple((Discrete(2), Box(0, 1, shape=(2,))))
    - (0, [0.1, 0.2]), (1, [0.25, 0.75]), ...
- ...

## Two spaces in environment
- action_space
- observation_space

In [5]:
print(f'action space: {env.action_space}\nsample: {env.action_space.sample()}')

action space: Discrete(2)
sample: 1


In [6]:
print(f'observation space: {env.observation_space}\nsample: {env.observation_space.sample()}')

observation space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
sample: [-1.4493227e+00 -3.6290537e+37  1.4106452e-01  2.6590647e+38]


In [7]:
print(f'return of env.reset: {env.reset()}')

return of env.reset: [-0.04666703  0.00728495 -0.02097982 -0.01013581]


In [8]:
print(f'return of env.step: {env.step(1)}')

return of env.step: (array([-0.04652133,  0.20270139, -0.02118254, -0.3093636 ], dtype=float32), 1.0, False, {})


# Training RL model

In [9]:
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose= 1)

Using cpu device


In [10]:
model.learn(total_timesteps= 10000)

-----------------------------
| time/              |      |
|    fps             | 2119 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1535        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008191365 |
|    clip_fraction        | 0.0969      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00516    |
|    learning_rate        | 0.0003      |
|    loss                 | 8.63        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0147     |
|    value_loss           | 59.5        |
-----------------------------------------
----------------------------------

# Save and reload the trained model

In [11]:
model_save_path = os.path.join('training', 'saved_model', 'PPO')

In [12]:
model.save(model_save_path)

In [13]:
del model

In [14]:
model = PPO.load(model_save_path, env=env)

IsADirectoryError: [Errno 21] Is a directory: 'training/saved_model/PPO'

# Evaluation

In [ ]:
evaluate_policy(model, env, n_eval_episodes= 5, render= True)

In [ ]:
env.close()

# Testing

In [ ]:
episodes = 5
for episode in range(episodes):
    state = env.reset() # initialize the observation (including state) of environment
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(state) # based on current state, predict the action
        observation = env.step(action) # take the action
        state, reward, done, info = observation # observation includes state, reward, done, info
        score += reward
    print(f'episode: {episode} score: {score}')
    env.close()

# Tensorboard

In [ ]:
log_save_path = os.path.join('training', 'log')
log_save_path

In [ ]:
env_name = "CartPole-v0"
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose= 1, tensorboard_log= log_save_path)

In [ ]:
model.learn(total_timesteps= 10000)

In [ ]:
!tensorboard --logdir={log_save_path}

# Callback

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold= 190, verbose= 1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best= stop_callback, # when there is a better model, call stop_callback
                             eval_freq= 10000, # evaluate every 10000 episodes
                             best_model_save_path= model_save_path,
                             verbose= 1)

In [ ]:
env_name = "CartPole-v0"
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose= 1)

In [ ]:
model.learn(total_timesteps= 20000, callback= eval_callback)

In [ ]:
# del model
best_model_save_path = os.path.join(model_save_path, 'best_model')
model = PPO.load(best_model_save_path, env= env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes= 10, render= True)

In [ ]:
env.close()